### Import Required Libraries and Set Up Environment Variables

In [3]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [4]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [6]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
built_url = (f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}"
             f"&end_date={end_date}&api-key={nyt_api_key}")

In [7]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19

for page in range(20):

    # create query with a page number
    paged_url = f"{built_url}&page={page}"
    
    # API results show 10 articles at a time

    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(paged_url)    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    if response.status_code == 200:
        reviews = response.json()

        # loop through the reviews["response"]["docs"] and append each review to the list
        if reviews["response"]["docs"]:
            for review in reviews["response"]["docs"]:
                reviews_list.append(review)
        # Print the page that was just retrieved
            print(f"Checked page {page}")
        else: 
        # Print the page number that had no results then break from the loop
            print(f"Page {page} had no results")
    else: 
        print(f"Error:{response.status_code}")
        break


Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [8]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [9]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()

df_reviews = pd.json_normalize(reviews_list)

df_reviews

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [10]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

def extract_title(headline):
    try:
        start = headline.index('\u2018') + 1
        end = headline.index('\u2019 Review') + len('\u2019 Review')
        return headline[start:end]
    except ValueError:
        return headline

def remove_leading_review(title):
    if title.startswith("Review: "):
        return title[len("Review: "):].strip()
    return title

df_reviews['title'] = df_reviews['headline.main'].apply(extract_title).apply(remove_leading_review)

df_reviews[['headline.main','title']]

,headline.main,title
0,"‘The Attachment Diaries’ Review: Love, Sick",The Attachment Diaries’ Review
1,Review: ‘What’s Love Got to Do With It?’ Proba...,‘What’s Love Got to Do With It?’ Probably a Lot
2,‘You Can Live Forever’ Review: Do You Love Me ...,You Can Live Forever’ Review
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,A Tourist’s Guide to Love’ Review
4,‘Other People’s Children’ Review: True Romance,Other People’s Children’ Review
...,...,...
195,Review: A Combustible Pair Find Love in ‘The O...,A Combustible Pair Find Love in ‘The Other Half’
196,"Review: Love as the World Wars, in ‘The Ottoma...","Love as the World Wars, in ‘The Ottoman Lieute..."
197,Review: It’s All Mirth and Taxes in ‘Love & Ta...,It’s All Mirth and Taxes in ‘Love & Taxes’
198,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...","‘Everybody Loves Somebody,’ a Rom-Com With Bite"


In [11]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):

    if isinstance(keyword_list, str):
        try:
            keyword_list = json.loads(keyword_list)
        except json.JSONDecodeError:
            return keyword_list

    
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item.get('value', 'N/A')};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords.strip()

# Fix the "keywords" column by converting cells from a list to a string

df_reviews['keywords'] = df_reviews['keywords'].apply(extract_keywords)

df_reviews[['keywords']].head()

,keywords
0,subject: Movies;creative_works: The Attachment...
1,"subject: Movies;persons: Kapur, Shekhar;person..."
2,subject: Movies;creative_works: You Can Live F...
3,subject: Movies;creative_works: A Tourist's Gu...
4,"subject: Movies;persons: Zlotowski, Rebecca;cr..."


In [12]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database

titles_list = df_reviews['title'].to_list()

print(titles_list[:10])

['The Attachment Diaries’ Review', '‘What’s Love Got to Do With It?’ Probably a Lot', 'You Can Live Forever’ Review', 'A Tourist’s Guide to Love’ Review', 'Other People’s Children’ Review', 'One True Loves’ Review', 'The Lost Weekend: A Love Story’ Review', 'A Thousand and One’ Review', 'Your Place or Mine’ Review', 'Love in the Time of Fentanyl’ Review']


### Access The Movie Database API

In [14]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [15]:
# Create an empty list to store the results

tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests

request_counter = 0

# Sanitize titles: 
def sanitize_title(title):
    # Replace curly quotes with straight quotes
    title = title.replace('’', "'").replace('‘', "'").replace('“', '"').replace('”', '"')
    # Remove the word 'Review' if present
    if ' Review' in title:
        title = title.replace(' Review', '')
    return title

# Loop through the titles

for title in df_reviews['title']:

    sanitized_title = sanitize_title(title)
    
    print(f"Searching for title: '{sanitized_title}'")
    
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(12)

    # Add 1 to the request counter
    request_counter += 1 
    
    # Perform a "GET" request for The Movie Database
    search_url = f"{url}{sanitized_title}{tmdb_key_string}"
    search_response = requests.get(search_url)

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try: 
        search_data = search_response.json()
        if search_data['results']:

        # Get movie id
            movie_id = search_data['results'][0]['id']

        # Make a request for a the full movie details
            details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
            details_response = requests.get(details_url)
            details_data = details_response.json()

        # Execute "GET" request with url
           
        # Extract the genre names into a list
            genres = [genre['name'] for genre in details_data['genres']]
           
        # Extract the spoken_languages' English name into a list
            spoken_languages = [language['english_name'] for language in details_data['spoken_languages']]

        # Extract the production_countries' name into a list
            production_countries = [country['name'] for country in details_data['production_countries']]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
            movie_info = {
                'title': sanitized_title,
                'genres': genres,
                'spoken_languages': spoken_languages,
                'production_countries': production_countries,
                'release_date':details_data.get('release_date'),
                'runtime':details_data.get('runtime'),
                'overview':details_data.get('overview')
            }
            tmdb_movies_list.append(movie_info)
        
        # Print out the title that was found
            print(f"Found {sanitized_title}")
        else:
            print(f"{sanitized_title} not found.")
            print(f"API Response: {search_data}")
    except Exception as e:
        print(f"Error occurred for movie {sanitized_title}:{e}")



Searching for title: 'The Attachment Diaries''
Found The Attachment Diaries'
Searching for title: ''What's Love Got to Do With It?' Probably a Lot'
'What's Love Got to Do With It?' Probably a Lot not found.
API Response: {'page': 1, 'results': [], 'total_pages': 1, 'total_results': 0}
Searching for title: 'You Can Live Forever''
Found You Can Live Forever'
Searching for title: 'A Tourist's Guide to Love''
Found A Tourist's Guide to Love'
Searching for title: 'Other People's Children''
Found Other People's Children'
Searching for title: 'One True Loves''
Found One True Loves'
Searching for title: 'The Lost Weekend: A Love Story''
Found The Lost Weekend: A Love Story'
Searching for title: 'A Thousand and One''
Found A Thousand and One'
Searching for title: 'Your Place or Mine''
Found Your Place or Mine'
Searching for title: 'Love in the Time of Fentanyl''
Found Love in the Time of Fentanyl'
Searching for title: 'Pamela, a Love Story''
Found Pamela, a Love Story'
Searching for title: 'In 

In [16]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

first_five_results = tmdb_movies_list[:5]

formatted_results = json.dumps(first_five_results,indent=4)

print(formatted_results)

[
    {
        "title": "The Attachment Diaries'",
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ],
        "release_date": "2021-10-07",
        "runtime": 102,
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship."
    },
    {
        "title": "You Can Live Forever'",
        "genres": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "English",
            "French"
        ],
        "

In [17]:
# Convert the results to a DataFrame

df_tmdb_movies = pd.DataFrame(tmdb_movies_list)

df_tmdb_movies

,title,genres,spoken_languages,production_countries,release_date,runtime,overview
0,The Attachment Diaries',"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],2021-10-07,102,"Argentina, 1970s. A desperate young woman goes..."
1,You Can Live Forever',"[Drama, Romance]","[English, French]","[Canada, United States of America]",2023-03-24,96,"When Jaime, a gay teenager, is sent to live in..."
2,A Tourist's Guide to Love',"[Romance, Comedy]","[English, Vietnamese]",[United States of America],2023-04-21,96,"After an unexpected break up, a travel executi..."
3,Other People's Children',"[Drama, Comedy]","[French, English]",[France],2022-09-21,104,"Rachel loves her life, her students, her frien..."
4,One True Loves',"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]",2023-04-07,100,Emma and Jesse are living the perfect life tog...
...,...,...,...,...,...,...,...
136,Sicilian Ghost Story',"[Mystery, Drama, Fantasy]",[Italian],"[France, Italy, Switzerland]",2017-11-18,122,In a little Sicilian village at the edge of a ...
137,The New Romantic',"[Drama, Comedy]",[English],[Canada],2018-10-19,82,In order to escape her looming post-graduation...
138,Pimp',[Drama],[English],[United States of America],2018-11-09,86,An urban love story set on the hard streets of...
139,In a Relationship',"[Drama, Comedy, Romance]",[English],"[Mexico, United States of America]",2018-11-09,92,Long-term couple Owen and Hallie are breaking ...


### Merge and Clean the Data for Export

In [19]:
# Merge the New York Times reviews and TMDB DataFrames on title

# Remove the last character from titles in df_reviews if it's an apostrophe
df_reviews['title'] = df_reviews['title'].apply(lambda x: x.replace(' Review', '')[:-1] if x.replace(' Review', '').endswith("'") or x.replace(' Review', '').endswith("’") else x.replace(' Review', '').strip())

# Inspect the sanitized titles in df_reviews
print("Sample titles from df_reviews:")
print(df_reviews['title'].head(10))

# Remove the last character from titles in df_tmdb_movies if it's an apostrophe
df_tmdb_movies['title'] = df_tmdb_movies['title'].apply(lambda x: x[:-1] if x.endswith("'") or x.endswith("’") else x)

# Inspect the sanitized titles in df_tmdb_movies
print("Sample titles from df_tmdb_movies:")
print(df_tmdb_movies['title'].head(10))

# Check for title standardization/ common titles 
common_titles = set(df_reviews['title']).intersection(set(df_tmdb_movies['title']))
print(len(common_titles))
common_titles_list = list(common_titles)[:10]

# Merge the DataFrames on 'titles'
merged_df = pd.merge(df_reviews, df_tmdb_movies, on='title', how='inner')

merged_df


Sample titles from df_reviews:
0                             The Attachment Diaries
1    ‘What’s Love Got to Do With It?’ Probably a Lot
2                               You Can Live Forever
3                          A Tourist’s Guide to Love
4                            Other People’s Children
5                                     One True Loves
6                     The Lost Weekend: A Love Story
7                                 A Thousand and One
8                                 Your Place or Mine
9                       Love in the Time of Fentanyl
Name: title, dtype: object
Sample titles from df_tmdb_movies:
0            The Attachment Diaries
1              You Can Live Forever
2         A Tourist's Guide to Love
3           Other People's Children
4                    One True Loves
5    The Lost Weekend: A Love Story
6                A Thousand and One
7                Your Place or Mine
8      Love in the Time of Fentanyl
9              Pamela, a Love Story
Name: title, dtyp

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries,release_date,runtime,overview
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],2021-10-07,102,"Argentina, 1970s. A desperate young woman goes..."
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,"[Drama, Romance]","[English, French]","[Canada, United States of America]",2023-03-24,96,"When Jaime, a gay teenager, is sent to live in..."
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]",2023-04-07,100,Emma and Jesse are living the perfect life tog...
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story,[Documentary],[English],[United States of America],2023-04-13,95,May Pang lovingly recounts her life in rock & ...
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One,"[Animation, Drama, Fantasy, Horror]",[Japanese],[Japan],1969-06-14,128,"Aldin, a vagabond water vendor, embarks of a s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,https://www.nytimes.com/2018/12/06/movies/the-...,"In Nathan Silver’s new film, American bros bre...",The New York Times,subject: Movies;creative_works: The Great Pret...,2018-12-06T12:00:01+0000,317,‘The Great Pretender’ Review: Tedious Love Aff...,None,None,The Great Pretender,...,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Great Pretender,"[Music, Documentary]",[English],[United States of America],2012-09-25,107,"Archive footage of interviews, concerts and pe..."
131,https://www.nytimes.com/2018/11/29/movies/sici...,Flipping from bucolic to brutal with distressi...,The New York Times,subject: Movies;creative_works: Sicilian Ghost...,2018-11-29T12:00:06+0000,258,‘Sicilian Ghost Story’ Review: Fairy-Tale Love...,Critic’s Pick,None,Love in an Enchanted Forest Turns Eerie,...,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Sicilian Ghost Story,"[Mystery, Drama, Fantasy]",[Italian],"[France, Italy, Switzerland]",2017-11-18,122,In a little Sicilian village at

In [20]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing

columns_to_fix = ['genres','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'", '"']


# Loop through the list of columns to fix
for col in columns_to_fix: 

    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)

    # Loop through characters to remove
    for char in characters_to_remove: 
        merged_df[col] = merged_df[col].str.replace(char, "")

# Display the fixed DataFrame

merged_df


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,byline.original,byline.person,byline.organization,title,genres,spoken_languages,production_countries,release_date,runtime,overview
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,2021-10-07,102,"Argentina, 1970s. A desperate young woman goes..."
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America",2023-03-24,96,"When Jaime, a gay teenager, is sent to live in..."
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America",2023-04-07,100,Emma and Jesse are living the perfect life tog...
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story,Documentary,English,United States of America,2023-04-13,95,May Pang lovingly recounts her life in rock & ...
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One,"Animation, Drama, Fantasy, Horror",Japanese,Japan,1969-06-14,128,"Aldin, a vagabond water vendor, embarks of a s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,https://www.nytimes.com/2018/12/06/movies/the-...,"In Nathan Silver’s new film, American bros bre...",The New York Times,subject: Movies;creative_works: The Great Pret...,2018-12-06T12:00:01+0000,317,‘The Great Pretender’ Review: Tedious Love Aff...,None,None,The Great Pretender,...,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Great Pretender,"Music, Documentary",English,United States of America,2012-09-25,107,"Archive footage of interviews, concerts and pe..."
131,https://www.nytimes.com/2018/11/29/movies/sici...,Flipping from bucolic to brutal with distressi...,The New York Times,subject: Movies;creative_works: Sicilian Ghost...,2018-11-29T12:00:06+0000,258,‘Sicilian Ghost Story’ Review: Fairy-Tale Love...,Critic’s Pick,None,Love in an Enchanted Forest Turns Eerie,...,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Sicilian Ghost Story,"Mystery, Drama, Fantasy",Italian,"France, Italy, Switzerland",2017-11-18,122,In a little Sicilian village at the edge of a ...
132,https://www.nytimes

In [21]:
# Drop "byline.person" column

print(merged_df.columns)

merged_df = merged_df.drop(columns =['byline.person']).reset_index(drop=True)

merged_df



Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title', 'genres', 'spoken_languages',
       'production_countries', 'release_date', 'runtime', 'overview'],
      dtype='object')


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,headline.sub,byline.original,byline.organization,title,genres,spoken_languages,production_countries,release_date,runtime,overview
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,None,By Jeannette Catsoulis,None,The Attachment Diaries,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,2021-10-07,102,"Argentina, 1970s. A desperate young woman goes..."
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,None,By Elisabeth Vincentelli,None,You Can Live Forever,"Drama, Romance","English, French","Canada, United States of America",2023-03-24,96,"When Jaime, a gay teenager, is sent to live in..."
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,None,By Brandon Yu,None,One True Loves,"Romance, Comedy, Drama","English, Spanish","Czech Republic, United States of America",2023-04-07,100,Emma and Jesse are living the perfect life tog...
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,None,By Glenn Kenny,None,The Lost Weekend: A Love Story,Documentary,English,United States of America,2023-04-13,95,May Pang lovingly recounts her life in rock & ...
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,None,By Manohla Dargis,None,A Thousand and One,"Animation, Drama, Fantasy, Horror",Japanese,Japan,1969-06-14,128,"Aldin, a vagabond water vendor, embarks of a s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,https://www.nytimes.com/2018/12/06/movies/the-...,"In Nathan Silver’s new film, American bros bre...",The New York Times,subject: Movies;creative_works: The Great Pret...,2018-12-06T12:00:01+0000,317,‘The Great Pretender’ Review: Tedious Love Aff...,None,None,The Great Pretender,...,None,By Glenn Kenny,None,The Great Pretender,"Music, Documentary",English,United States of America,2012-09-25,107,"Archive footage of interviews, concerts and pe..."
131,https://www.nytimes.com/2018/11/29/movies/sici...,Flipping from bucolic to brutal with distressi...,The New York Times,subject: Movies;creative_works: Sicilian Ghost...,2018-11-29T12:00:06+0000,258,‘Sicilian Ghost Story’ Review: Fairy-Tale Love...,Critic’s Pick,None,Love in an Enchanted Forest Turns Eerie,...,None,By Jeannette Catsoulis,None,Sicilian Ghost Story,"Mystery, Drama, Fantasy",Italian,"France, Italy, Switzerland",2017-11-18,122,In a little Sicilian village at the edge of a ...
132,https://www.nytimes.com/2018/11/08/movies/the-...,A college newspaper columnist writes about her...,The New York Times,subject: Movies;creative_works: The New Romant...,2018-11-08T12:05:01+0000,269,‘The New Romantic’ Review: Experiments With Lo...,None,None,The New Romantic,...,None,By Teo Bugbee,None,The New Romantic,"Drama, Comedy",English,Canad

In [22]:
# Delete duplicate rows and reset index

merged_df = merged_df.drop_duplicates().reset_index(drop=True)

In [23]:
# Export data to CSV without the index

merged_df.to_csv('merged_data.csv', index=False)